In [15]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["LLM_AUTH_KEY"] = "TEST"
# load env from .env file in ../
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")


True

In [17]:
# os.remove("tasks_1.pkl")

In [18]:
# from coding.finetune.pipeline import FinetunePipeline
# from coding.rewards.codesim import CodeSimModel
# from coding.finetune.llm.manager import LLMManager
# config = type('Config', (), {'finetune_test_size': 1})()
# code_sim_model = CodeSimModel()

# pipeline = FinetunePipeline(config=config, code_sim_model=code_sim_model)

# pipeline.load_tasks()
# print("Gotten tasks")
# # llm_manager = LLMManager()

In [19]:
from coding.tasks.swe import SWEBenchTask
from coding.schemas.context import Context
task = SWEBenchTask(llm=None,context=Context(
    extras={"base_commit": "336043183333c8078bbd035f5058ef91c5423c30", "pull_number": 77},
    title="dragnet-org/dragnet",
    content="diff --git a/requirements.txt b/requirements.txt\nindex 15a6934..65d3dcf 100644\n--- a/requirements.txt\n+++ b/requirements.txt\n@@ -2,7 +2,7 @@ nose\n coverage\n Cython>=0.21.1\n lxml\n-scikit-learn>=0.15.2,<=0.19.2\n+scikit-learn>=0.15.2,<=0.20\n numpy\n scipy\n ftfy>=4.1.0,<5.0.0\ndiff --git a/setup.py b/setup.py\nindex ebed646..366d298 100644\n--- a/setup.py\n+++ b/setup.py\n@@ -98,7 +98,7 @@ def find_libxml2_include():\n     install_requires=[\n         'Cython>=0.21.1',\n         'lxml',\n-        'scikit-learn>=0.15.2,<=0.19.2',\n+        'scikit-learn>=0.15.2,<=0.20',\n         'numpy',\n         'scipy',\n         'ftfy>=4.1.0,<5.0.0'\n",
    topic="bump scikit-learn version ceiling to 0.20.1?\nHi! `dragnet` currently requires `scikit-learn<=0.19.2`, which prevents users from developing alongside the final Py2/3 compatible release of 0.20.1 (see [here](http://scikit-learn.org/dev/whats_new.html#version-0-20-0)). Would you be able to bump the version ceiling and publish a new release? Assuming no version-driven incompatibilities, I _think_ this shouldn't be too much work; you don't have to train new, version-specific models, right?\nbump scikit-learn version ceiling to 0.20.1?\nHi! `dragnet` currently requires `scikit-learn<=0.19.2`, which prevents users from developing alongside the final Py2/3 compatible release of 0.20.1 (see [here](http://scikit-learn.org/dev/whats_new.html#version-0-20-0)). Would you be able to bump the version ceiling and publish a new release? Assuming no version-driven incompatibilities, I _think_ this shouldn't be too much work; you don't have to train new, version-specific models, right?\n",
))

In [114]:
import os

logic = {}
test_submission_dir = "example-submission"

# Read all files in test-submission directory
for root, dirs, files in os.walk(test_submission_dir):
    # Skip __pycache__ directories
    if '__pycache__' in dirs:
        dirs.remove('__pycache__')
        
    # Get relative path from test_submission_dir
    rel_path = os.path.relpath(root, test_submission_dir)
    
    # Process all files in current directory
    for filename in files:
        # Skip __pycache__ files
        if '__pycache__' in filename:
            continue
            
        file_path = os.path.join(root, filename)
        # Get the relative path for the logic dict key
        if rel_path == '.':
            logic_key = filename
        else:
            logic_key = os.path.join(rel_path, filename)
            
        with open(file_path, 'r', encoding='latin-1') as f:
            logic[logic_key] = f.read()

In [115]:
from coding.schemas.tracking import TrackingInfo

tracking_info = TrackingInfo(
    logic=logic,
    block=0,
    hotkey="test",
    uid=0,
    results=[]
)


In [116]:
# Create repo_files dict from task.repo.path
repo_files = {}

# Walk through all files in repo path
for root, dirs, files in os.walk(task.repo.path):
    # Skip __pycache__ directories
    if '__pycache__' in dirs:
        dirs.remove('__pycache__')
        
    # Get relative path from repo root
    rel_path = os.path.relpath(root, task.repo.path)
    
    # Process all files
    for filename in files:
        # Skip __pycache__ files
        if '__pycache__' in filename:
            continue
            
        file_path = os.path.join(root, filename)
        
        # Get the relative path for the repo_files dict key
        if rel_path == '.':
            repo_key = filename
        else:
            repo_key = os.path.join(rel_path, filename)
            
        # Read file contents
        with open(file_path, 'r', encoding='latin-1') as f:
            repo_files[repo_key] = f.read()


In [117]:
from coding.finetune.pipeline import verify_logic

pass_logic, pass_msg = verify_logic(tracking_info.logic)

pass_logic, pass_msg

(False, 'Disallowed module: swebase')

In [118]:
from coding.finetune.dockerutil import build_docker_container, run_docker_container

build_docker_container(tracking_info.logic, tracking_info.hotkey, repo_files=repo_files)

Wrote /tmp/tmpp7mqi7b3/fix.py
Wrote /tmp/tmpp7mqi7b3/search.py
Wrote /tmp/tmpp7mqi7b3/submission.py
Wrote /tmp/tmpp7mqi7b3/diff.py
Wrote /tmp/tmpp7mqi7b3/files.py
Wrote repo file /tmp/tmpp7mqi7b3/repo/.travis.yml
Wrote repo file /tmp/tmpp7mqi7b3/repo/MANIFEST.IN
Wrote repo file /tmp/tmpp7mqi7b3/repo/setup.py
Wrote repo file /tmp/tmpp7mqi7b3/repo/requirements.txt
Wrote repo file /tmp/tmpp7mqi7b3/repo/.gitignore
Wrote repo file /tmp/tmpp7mqi7b3/repo/README.md
Wrote repo file /tmp/tmpp7mqi7b3/repo/Vagrantfile
Wrote repo file /tmp/tmpp7mqi7b3/repo/provision.sh
Wrote repo file /tmp/tmpp7mqi7b3/repo/Makefile
Wrote repo file /tmp/tmpp7mqi7b3/repo/LICENSE
Wrote repo file /tmp/tmpp7mqi7b3/repo/dragnet_www2013.pdf
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/index
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/HEAD
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/FETCH_HEAD
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/description
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/config
Wrote repo file /tmp/tmpp7mqi7

Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/33/6043183333c8078bbd035f5058ef91c5423c30
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/31/6edaa7fde07273a2be6ad5a8731243a54ed277
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/pack/pack-8ea5a795892cf3afa21e8af4ab1d3d606e8bbe10.pack
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/pack/pack-8ea5a795892cf3afa21e8af4ab1d3d606e8bbe10.idx
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/3e/7901cbc936dc0614817cb313c57cb03f170f46
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/c5/9104502278ccce4f137712e9c397ebdeab4e25
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/9c/e832118b3b405c4d8d88514ca90efc4b0f4dae
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/15/a69343276da9e8bf4ba7b4aec4d4f43e05d812
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/82/076056ec0fd770c0fd50542fb52f052dd374f0
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/objects/47/31e28d0759abf93a575fb51391b8b0768fca1e
Wrote repo file /tmp/tmpp7mqi7b3/repo/.git/

'sha256:56808218b7922b26e120e4c5761b9c3d3d6d313a2062e9ea35f2ea7c9fb8321c'

In [119]:
f"swe-logic-{str(tracking_info.hotkey)}-1"

'swe-logic-test-1'

In [120]:
# task = pipeline.tasks[0]
container = run_docker_container(
        f"swe-logic-{str(tracking_info.hotkey)}-1",
        task.repo,
        tracking_info.hotkey,
    )
# print the container logs
container.logs()


b''

In [121]:
import time 
time.sleep(4)  
import requests
response = requests.post(
            f"http://{os.environ['DOCKER_HOST_IP']}:3000/call",
            json={
                "repo_location": "/app/repo",
                "issue_description": task.query,
            },
            timeout=360,
        )
response.json()

{'result': {'edits': [{'file_name': 'setup.py',
    'line_number': 101,
    'line_content': "       'scikit-learn>=0.15.2,<=0.19.2',",
    'new_line_content': "       'scikit-learn>=0.15.2,<=0.20.1',"}]}}

In [122]:
from coding.schemas import Patch
from coding.tasks.swe import chunk_patch
chunk_patch(Patch(**response.json()["result"]))

[PatchChunk(file_name='setup.py', start_index=101, end_index=101, content="       'scikit-learn>=0.15.2,<=0.19.2',", new_content="       'scikit-learn>=0.15.2,<=0.20.1',")]

In [123]:
from coding.schemas import Patch
patch = Patch(**response.json()["result"])
score = task.score(patch, 100)
score

valid patch edits=[Edit(file_name='requirements.txt', line_number=5, line_content='scikit-learn>=0.15.2,<=0.19.2', new_line_content=''), Edit(file_name='requirements.txt', line_number=5, line_content='', new_line_content='scikit-learn>=0.15.2,<=0.20'), Edit(file_name='setup.py', line_number=101, line_content="'scikit-learn>=0.15.2,<=0.19.2',", new_line_content=''), Edit(file_name='setup.py', line_number=101, line_content='', new_line_content="'scikit-learn>=0.15.2,<=0.20',")]
percent lines in miner 0.25
valid chunks [PatchChunk(file_name='requirements.txt', start_index=5, end_index=5, content='scikit-learn>=0.15.2,<=0.19.2', new_content='scikit-learn>=0.15.2,<=0.20'), PatchChunk(file_name='setup.py', start_index=101, end_index=101, content="'scikit-learn>=0.15.2,<=0.19.2',", new_content="'scikit-learn>=0.15.2,<=0.20',")]
old miner chunk content        'scikit-learn>=0.15.2,<=0.19.2',
old valid chunk content 'scikit-learn>=0.15.2,<=0.19.2',
new miner chunk content        'scikit-learn>=

0.375

In [124]:
llm_manager.get_count()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 llm_manager.get_count()                                                                      │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'llm_manager' is not defined

In [48]:
import requests
import traceback
from coding.schemas import Patch
from coding.constants import COMPETITION_ID
from coding.schemas.tracking import TaskResult
from time import sleep
for task in pipeline.tasks:
    llm_manager.init_key(tracking_info.hotkey)
    container = run_docker_container(
        f"swe-logic-{str(tracking_info.hotkey)}-{COMPETITION_ID}",
        task.repo,
        tracking_info.hotkey,
    )
    sleep(2)
    print("reaching out to container now")
    try:
        response = requests.post(
            f"http://localhost:3000/call",
            json={
                "repo_location": "/app/repo",
                "issue_description": task.query,
            },
            timeout=360,
        )
        print(response.json())
        response.raise_for_status()
        result = response.json()["result"]
        patch = Patch(**result)
        print("test")
        print(patch)
        score = task.score(patch, llm_manager.get_count())
        llm_manager.reset_count()
    except Exception as e:
        print(f"Request failed: {e}")
        print(traceback.format_exc())
        score = 0

    tracking_info.results.append(TaskResult(
        logic=logic,
        commit_hash=task.base_commit,
        score=score,
    ))

tracking_info.results


reaching out to container now


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:17                                                                                   │
│                                                                                                  │
│   14 │   sleep(2)                                                                                │
│   15 │   print("reaching out to container now")                                                  │
│   16 │   try:                                                                                    │
│ ❱ 17 │   │   response = requests.post(                                                           │
│   18 │   │   │   f"http://localhost:3000/call",                                                  │
│   19 │   │   │   json={                                                                          │
│   20 │   │   │   │   "repo_location": "/app/repo",                                               │
│                                                                                                  │
│ /home/bskesse/miniconda3/envs/code311/lib/python3.11/site-packages/requests/api.py:115 in post   │
│                                                                                                  │
│   112 │   :rtype: requests.Response                                                              │
│   113 │   """                                                                                    │
│   114 │                                                                                          │
│ ❱ 115 │   return request("post", url, data=data, json=json, **kwargs)                            │
│   116                                                                                            │
│   117                                                                                            │
│   118 def put(url, data=None, **kwargs):                                                         │
│                                                                                                  │
│ /home/bskesse/miniconda3/envs/code311/lib/python3.11/site-packages/requests/api.py:59 in request │
│                                                                                                  │
│    56 │   # avoid leaving sockets open which can trigger a ResourceWarning in some               │
│    57 │   # cases, and look like a memory leak in others.                                        │
│    58 │   with sessions.Session() as session:                                                    │
│ ❱  59 │   │   return session.request(method=method, url=url, **kwargs)                           │
│    60                                                                                            │
│    61                                                                                            │
│    62 def get(url, params=None, **kwargs):                                                       │
│                                                                                                  │
│ /home/bskesse/miniconda3/envs/code311/lib/python3.11/site-packages/requests/sessions.py:589 in   │
│ request                                                                                          │
│                                                                                                  │
│   586 │   │   │   "allow_redirects": allow_redirects,                                            │
│   587 │   │   }                                                                                  │
│   588 │   │   send_kwargs.update(settings)                                                       │
│ ❱ 589 │   │   resp = self.send(prep, **send_kwargs)                                              │
│   590 │   │                                                                                      │
│   591 │   │   return resp                                  